<a href="https://colab.research.google.com/github/grand00father/Itmo_MS/blob/main/ITMO_MS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata
GROQ_KEY = userdata.get("GROK_KEY")

In [2]:
!pip install groq faiss-cpu==1.13.2 langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 9.9 MB/s eta 0:00:00


In [3]:
from typing import TypedDict, List, Optional
import json, re
from langgraph.graph import StateGraph, END

In [4]:
from groq import Groq
client = Groq(api_key=GROQ_KEY)

In [5]:
class Interview_states(TypedDict):
  name: Optional[str]
  position: Optional[str]
  grade: Optional[str]
  experience: Optional[str]
  history_user: List[str]
  history_int_obs: List[str]
  history_think: List[str]
  answer: Optional[str]
  done: bool
  final_feedback : Optional[str]
  turns: List[dict]
  id: int

In [19]:
def intervier(state: Interview_states):
  if state['id'] == -1:
    messages = [
        {"role": "system", "content": (
            '''Твой ответ должен содержать ТОЛЬКО формат JSON, который выглядит как:\n
            {\n
            "question":string\n
            }\n
            Ты IT специалист по имени Петр и ты начинаешь проводить техническое интервью у кандидата.\n
            Твоя задача:\n
            Поприветствовать кандидата и четко узнать данные о нем.
            '''
        )}
        ]
    response = client.chat.completions.create(
              model='qwen/qwen3-32b',
              messages=messages,
              reasoning_effort="default")

    msg = response.choices[0].message.content.strip()
    match = re.search(r"\{[\s\S]*\}", msg)
    res = json.loads(match.group())
    print(res['question'])
    return {
       'history_user' : state['history_user'] + [f"Вопрос: {res['question']}"]}

  if state['done']:
    messages = [
        {"role": "system", "content": (
            '''Твой ответ должен содержать ТОЛЬКО формат JSON, который выглядит как:\n
            {\n
            "think_about":string,\n
            "summary":string\n
            }\n
            Ты IT специалист и это конец технического интервью.\n
            Твоя задача:\n
            Подумай, что ты думаешь о рекомендациях эксперта.\n
            Исходя из рекомендаций дать кандидату отзыв о его выступлении, который будет записан в summary и должен включать:\n
            Вердикт - Уровень кандидата (Junior / Middle / Senior) на основе предыдущих ответов (если кандидат не справлялся вообще, то поставь нулевой уровень);\n
            Cписок тем, затронутых в интервью - Темы, где кандидат дал точные ответы  и Темы, где были допущены ошибки или кандидат сказал «не знаю» (ты должен добавить правильный ответ на эти вопросы).
            Анализ Soft Skills & Communication на основании того, насколько кандидат понятно излагает мысли, пытался ли кандидат выкрутиться/соврать или честно признал незнание;\n
            Персональный Roadmap - Список конкретных тем/технологий, которые нужно подтянуть (на основе выявленных пробелов).
            '''
        )},
        {"role": "user", "content": f"Рекомендации эксерта:{state["history_int_obs"]}, Последний ответ:{state['answer']}"}
        ]
    response = client.chat.completions.create(
              model='qwen/qwen3-32b',
              messages=messages,
              reasoning_effort="default")

    msg = response.choices[0].message.content.strip()
    match = re.search(r"\{[\s\S]*\}", msg)
    res = json.loads(match.group())
    print(f"{res['summary']}")
    return {
        'final_feedback' :  [f"Резюме: {res['summary']}"],
        'history_user' : state['history_user'] + [f"Резюме: {res['summary']}"],
        'history_think': state['history_think'] + [f"{res['think_about']}"],
        "turns": state["turns"] + [{
              "turn_id": state["id"],
              "agent_visible_message": state["history_user"][-2],
              "user_message": state["answer"],
              "internal_thoughts": f"[Observer]:<{state['history_int_obs'][-1]}> \n [Interviewer]:<{res['think_about']}>"}]}

  messages = [
        {"role": "system", "content": (
            '''Твой ответ должен содержать ТОЛЬКО формат JSON, который выглядит как:\n
            {\n
            "think_about":string,\n
            "question":string\n
            }\n
            Ты IT специалист по имени Петр и проводишь техническое интервью на позицию''' + state['position'] + '''у кандидата по имени'''+ state['name'] + ''' с заявленным уровнем''' + state['grade']+ ''' и с опытом''' + state['experience'] +'''.\n
            Твоя задача:\n
            Подумай, что ты думаешь о рекомендациях эксперта.\n
            Вести с кандидатом диалог;\n
            Задавать кандидату вопросы, которые соответствуют его подготовке и уровню знаний;\n
            Также ориентируйся на рекомендации.
            '''
        )},
        {"role": "user", "content": f"Рекомендации эксерта:{state["history_int_obs"]}, Последний ответ:{state['answer']}"}
    ]
  response = client.chat.completions.create(
            model='qwen/qwen3-32b',
            messages=messages,
            reasoning_effort="default")

  msg = response.choices[0].message.content.strip()
  match = re.search(r"\{[\s\S]*\}", msg)
  res = json.loads(match.group())
  print(res['question'])
  if state['id'] != 0:
    return {'history_user': state['history_user'] + [f"Вопрос: {res['question']}"],
            'history_think': state['history_think'] + [f"{res['think_about']}"],
            "turns": state["turns"] + [{
              "turn_id": state["id"],
              "agent_visible_message": state["history_user"][-2],
              "user_message": state["answer"],
              "internal_thoughts": f"[Observer]:<{state['history_int_obs'][-1]}> \n [Interviewer]:<{res['think_about']}>"}]}
  return {'history_user': state['history_user'] + [f"Вопрос: {res['question']}"],
            'history_think': state['history_think'] + [f"{res['think_about']}"]}


In [20]:
def observer(state: Interview_states):
  if state['id'] == 0:
    messages = [
        {"role": "system", "content": (
            '''Твой ответ должен содержать ТОЛЬКО формат JSON, который выглядит как:\n
            {\n
            "name":string,\n
            "position":string,\n
            "grade":string,\n
            "experience":string,\n
            "recomendation":string\n
            }\n
            Ты IT эксперт, который помогает проводить техническое интервью.\n
            Твоя задача:\n
            Кандидат приставился, тебе необходимо выделить из его ответа соответствующие данные, которые указаны в json;\n
            Дай рекомендации интервьеру, по каким темам лучше дальше задать вопрос.
            '''
        )},
        {"role": "user", "content": f"История диалога:{state["history_user"]}, Диалог с интервьером: {state['history_int_obs']}, Последний ответ:{state['answer']}"}
        ]
    response = client.chat.completions.create(
              model='qwen/qwen3-32b',
              messages=messages,
              reasoning_effort="default")

    msg = response.choices[0].message.content.strip()
    match = re.search(r"\{[\s\S]*\}", msg)
    res = json.loads(match.group())
    return {
       'name' : res['name'],
       'position' : res['position'],
       'grade' : res['grade'],
       'experience' : res['experience'],
       'history_int_obs': state['history_int_obs'] + [f"Рекомендация: {res['recomendation']}"]}

  messages = [
        {"role": "system", "content": (
            '''Твой ответ должен содержать ТОЛЬКО формат JSON, который выглядит как:\n
            {\n
            "recomendation": string,\n
            "done": bool,\n
            "summary": string\n
            Ты IT эксперт, который помогает проводить техническое интервью на позицию''' + state['position'] + '''у кандидата по имени'''+ state['name'] + ''' с заявленным уровнем''' + state['grade']+ ''' и с опытом''' + state['experience'] +'''.\n
            Твоя задача:\n
            В начале посоветовать интервьеру поздороваться с кандидатом и узнать информацию о нем;\n
            Оценить ответ кандидата и решить, какой вопрос лучше задать дальше;\n
            В случае, если ты понимаешь, что кандидат не может справиться с вопросом, порекомендуй снизить сложность вопросов или, наоборот, повысить:\n
            Решать, достаточно ли информации;\n
            Если информации недостаточно — задать следующий логичный вопрос;\n
            Если информации достаточно — порекомендуй завершить интервью.\n

            }'''
        )},
        {"role": "user", "content": f"История диалога:{state["history_user"]}, Диалог с интервьером: {state['history_int_obs']}, Последний ответ:{state['answer']}"}
    ]
  response = client.chat.completions.create(
            model='qwen/qwen3-32b',
            messages=messages,
            reasoning_effort="default")

  msg = response.choices[0].message.content.strip()

  match = re.search(r"\{[\s\S]*\}", msg)
  res = json.loads(match.group())
  if res['done']:
    return {
        'history_int_obs': state['history_int_obs'] + [f"Резюме: {res['summary']}"],

        'done': True
    }

  return {
      'history_int_obs': state['history_int_obs'] + [f"Рекомендация: {res['recomendation']}"],

      'done': False
  }

In [8]:
def user_answer(state: Interview_states):
  ans_user = input('>>> ')
  return {
      'answer': ans_user,
      'id' : state['id'] + 1,
      'history_user' : state["history_user"] + [f"Ответ: {state['answer']}"]
  }

In [9]:
def route(state: Interview_states):
  return END if state['done'] == True else 'answer'

In [21]:
graph = StateGraph(Interview_states)

graph.add_node('intervier', intervier)
graph.add_node('observer', observer)
graph.add_node('answer', user_answer)

graph.set_entry_point('intervier')

graph.add_edge('observer', 'intervier')
graph.add_edge('answer', 'observer')

graph.add_conditional_edges(
    'intervier',
    route,
    {
        "answer": "answer",
        END: END
    }
)

app = graph.compile()

In [35]:
state = {
    'name':None,
    'position': None,
    'grade': None,
    'experience': None,
    'history_user': [],
    'history_int_obs': [],
    'history_think' : [],
    'answer': None,
    'done': False,
    'turns': [],
    'id': -1,
    'final_feedback' : None
}

state = app.invoke(state)
logs = {
    "participant_name":state['name'],
    'turns': state['turns'],
    'final_feedback':state['final_feedback']
}
with open("interview_log.json", "w", encoding="utf-8") as f:
    json.dump(logs, f, ensure_ascii=False, indent=4)

Здравствуйте, меня зовут Петр. Рад приветствовать вас на техническом интервью. Пожалуйста, представьтесь: как зовут, сколько опыта работы в IT и на каком уровне говорите по-английски (включая IT-английский)?
>>> Привет, меня зовут Святослав. Я хочу попасть на вакансию стажер ML инженер.с
Расскажите, пожалуйста, чем отличаются алгоритмы супервизированного машинного обучения от несупервизированного. Приведите по одному примеру задачи и алгоритма для каждой категории.
>>> 1 - когда у нас есть известный таргет и мы обучаем модель на нем (логистическая регрессия). 2 - когда модель учится относить обьекты к таргету не зная его (метод k близжайших соседей)
Какие метрики вы используете для оценки качества моделей в задачах классификации и регрессии? Приведите пример выбора метрики под конкретную задачу.
>>> recall, precision, auc-roc . регрессия - mse,mae, усредненный mae по таргету. бинарная классификации больных - recall
В вашем примере с задачей бинарной классификации больных вы выбрали rec